In [ ]:
--Austin's Shit



In [1]:
require('hdf5')
require 'nn'
require 'optim'
require 'math'

f = hdf5.open("SST1.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
Y_valid = f:read("valid_output"):all()
--X_test = f:read("test_input"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

# Doc by word

In [2]:
--DOC BY WORD (vincent's stuff)
--feature weight: counts
function createDocWordMatrix(vocab, max_sent_len, sparseMatrix)
    docword = torch.zeros(sparseMatrix:size(1), vocab)
    for i=1,sparseMatrix:size(1) do
        for j=1, max_sent_len do
            local idx = (sparseMatrix[i][j])
            if idx ~= 0 then
                docword[i][idx] = 1 --+ docword[i][idx]
            end
        end        
    end
    return docword
end

function onehotencode(classes, target)
    onehot = torch.zeros(target:size(1), classes)
    for i=1,target:size(1) do
        onehot[i][target[i]] = 1
    end
    return onehot
end

In [3]:
X_train=createDocWordMatrix(nfeatures, 53, X_train)
X_valid=createDocWordMatrix(nfeatures, 53, X_valid)

In [4]:
W0 = torch.randn(5,nfeatures)*.01
b0 =torch.zeros(5,1)*.001

In [ ]:
X_toy = torch.Tensor({{1,0,0},{0,1,0},{0,0,1},{1,0,0},{1,0,0},{0,1,0},{0,0,1},{1,0,0},{1,0,0},{0,1,0},{0,0,1},{1,0,0}})
Y_toy = torch.Tensor({1,2,3,1,1,2,3,1,1,2,3,1})
W_toy = torch.zeros(5,3)
b_toy = torch.zeros(5,1)*.001
print(X_toy)

In [ ]:
print(X_valid:sub(1,1,1,50))


In [61]:
function isnan(m)
    if m ~= m then
        return true
    else
        return false
    end
end

function linear(b, W, x)
    --weight matrix W (dout x din), bias b (dout x 1), input matrix x (n x din)
    local B = b:clone()
    local B = B:expand(B,B:size(1),x:size(1))
    return ((W*x:t()):add(B)):t()
end
    
function second_max(Y_pred, Y_true)
    --Y_pred is a nx5 tensor
    local second_maxes = torch.zeros(Y_pred:size(1),1)
    local indices = second_maxes:clone()
    local Y_prime = (Y_pred):clone()
    for i = 1, Y_pred:size(1) do
        Y_prime[i][Y_true[i]] = -100
        z1,z2 = torch.max(Y_prime[i],1)
        second_maxes[i] = z1[1]
        indices[i] = z2[1]
    end
    return second_maxes, indices
end

function correct_val(Y_pred, Y_true)
    return_val = torch.zeros(Y_true:size(1),1)
    for i = 1, Y_true:size(1) do
        return_val[i] = Y_pred[i][Y_true[i]]
    end
    return return_val
end


function hinge_grad(W,x,b,Y_true,lambda)
    local Y_pred = linear(b,W,x)
    local condition = torch.Tensor(Y_true:size(1),1)
    local correct = correct_val(Y_pred,Y_true) 
    local second, s_indices = second_max(Y_pred,Y_true)
    local condition = correct - second
    local W_grad = torch.zeros(W:size())
    local W_temp = torch.zeros(W:size())
    local b_grad = torch.zeros(b:size())
    local b_temp = torch.zeros(b:size())
    local debug = torch.zeros(4)
    for i = 1, condition:size(1) do
        if condition[i][1] < 1 then
            for j = 1, W:size(1) do
                if j == Y_true[i] then
                    b_temp[j] = -1
                    W_temp[j] = x[i]*(-1)
                    debug[1] = debug[1] + 1
                elseif j == s_indices[i][1] then
                    debug[2] = debug[2] + 1
                    b_temp[j] = 1
                    W_temp[j] = x[i]
                else
                    debug[3] = debug[3] + 1
                    b_temp[j] = 0
                    W_temp[j] = W_temp[j]*0
                end
            end
            b_grad:add(b_temp)
            W_grad:add(W_temp)
        else
            debug[4] = debug[4] + 1
            b_temp = torch.zeros(b_temp:size())
            W_temp = torch.zeros(W_temp:size())
        end
    end
    return W_grad:add(2*lambda,W),b_grad:add(2*lambda,b)
    --return W_grad,0--b_grad
end

function ReLU(input)
    for i = 1,input:size(1) do
        if input[i][1]<0 then
            input[i][1] =0
        end
    end
    return input
end

function hinge_loss(Y_pred, Y_true, W,lambda,b)
    --Y_pred is a nx5 tensor, Y_true is an nx1 tensor
    params = torch.cat(W,b,2)
    local ones_vector = torch.ones(Y_true:size(1),1)
    --local loss_vector = nn.ReLU():forward(ones_vector+second_max(Y_pred, Y_true)-correct_val(Y_pred,Y_true))
    local loss_vector = ReLU(ones_vector+second_max(Y_pred, Y_true)-correct_val(Y_pred,Y_true))
    loss1 = lambda*torch.pow((torch.norm(params)),2)
    loss2 = loss_vector:sum(1)[1][1]
    
    return (loss1 + loss2)
end

function obj(params,X,Y,batch,lambda)
    if batch > 0 then
        bsize= batch
        local idx = torch.randperm(X:size(1)):sub(1,bsize)

        x = torch.Tensor(bsize, X:size(2))

        Y = Y:reshape(Y:size(1), 1)

        y = torch.Tensor(bsize, 1)

        for i=1,bsize do
            x[i] = X[idx[i]]
            y[i] = Y[idx[i]]
        end
        
        y = y:squeeze()
    else
        y = Y
        x = X
    end
    
    local w = (params:sub(1,params:size(1),1,params:size(2)-1))
    
   
    b = params:sub(1,params:size(1),params:size(2),params:size(2))
    local ypred = linear(b,w,x)
    local loss = hinge_loss(ypred,y,w,lambda,b)
    
    local w1,b1 = hinge_grad(w,x,b,y,lambda)
    local grads = torch.zeros(params:size())
    grads[{{},{1,params:size(2)-1}}] = w1
    grads[{{},{params:size(2),params:size(2)}}] = b1
    return loss, grads
end

function obj_final(params)
    
    loss,grads = obj(params, X_train, Y_train, 20000, 1)
    print(loss)
    return loss,grads
end

In [ ]:
p0 = torch.cat(W0,b0,2)
_,_,n = optim.checkgrad(obj_final,p0)
print(n:size())

In [ ]:
toy_pred = linear(b_toy,W_toy,X_toy)

toy_second, toy_indices = second_max(toy_pred, Y_toy)
loss = hinge_loss(toy_pred, Y_toy, W_toy, 0,b_toy)
print(d)

In [65]:
--p0 = torch.cat(W0,b0,2)
local state = {maxIter = 1, learningRate = .001, evalCounter = 1}
d = torch.Tensor(300)
for i = 1,d:size(1) do
    p0, m = optim.adam(obj_final,p0, state)
    d[i] = m[1]
end
print(d)

4081.9669779339	


4126.616425437	


4168.1246791406	


4081.4470523632	


4185.0348938664	


...

4059.2665176628	


4031.7482486836	


3938.9614117393	


4105.7612197062	


 4081.9670
 4126.6164
 4168.1247
 4081.4471
 4185.0349
 4144.4596
 4125.5491
 4147.3621
 3987.0877
 4136.9464
 4117.0390
 4129.0206
 4238.3940
 4151.6042
 4038.4816
 4119.6189
 4108.6259
 4076.3367
 4115.6236
 4121.6519
 4237.8635
 4089.0224
 4142.0749
 4074.7259
 4078.5210
 4060.1084
 4003.0459
 4020.7418
 4028.0993
 4057.8975
 4140.3664
 4031.7865
 4063.5472
 4157.0385
 4149.0376
 3995.3455
 4029.8823
 4090.7216
 4126.7806
 4057.4358
 4039.6133
 4127.8967
 3948.5553
 4019.2571
 4034.9212
 4073.0351
 4059.8102
 4020.6988
 4078.6803
 4078.0400
 4019.5818
 4023.6140
 4077.2529
 4041.9263
 4094.9480
 4037.1895
 4096.6394
 4046.3039
 4085.6561
 4025.6434
 4076.1870
 4126.2074
 4101.0942
 4084.3568
 4131.0181
 4054.6770
 4051.4075
 4025.4100
 4084.4575
 4005.9318
 4106.9844
 4070.0928
 4076.2107
 4055.8489
 4004.2001
 3991.6639
 4013.3737
 4079.4183
 4045.8676
 4054.6654
 4031.8301
 3970.7267
 3941.6262
 4043.8071
 3959.5882
 4021.8423
 4034.8679
 4064.5799
 4058.5263
 4055.4841
 4003.4802

In [66]:

 --intercept
b = p0:sub(1, p0:size(1), p0:size(2), p0:size(2))

--coefficients
W = p0:sub(1, p0:size(1), 1, p0:size(2)-1)

function predict(X, W, b)
    local b = b:t()
    return (X*W:t()):add(b:expand(b, X:size(1), b:size(2)))
end
function predict_score(ypred, ytrue)
    local c = 0
    for i=1,ypred:size(1) do
        if ypred[i][1] == ytrue[i] then
            c = c + 1       
        end
    end
    return c/ypred:size(1)
end


In [67]:
Y_pred = predict(X_valid, W, b)
_, score = torch.max(Y_pred, 2)
--_,Y_true = torch.max(Y_valid, 2)
acc_score = predict_score(score, Y_valid)
print(acc_score)

0.3905540417802	


In [ ]:
score

In [ ]:
print(W:sub(1,5,1,10))

In [ ]:
f,f_old = 1,2

In [ ]:
z = torch.sort(Y_train)

In [ ]:
z[40000]

In [ ]:
930-660

In [ ]:
z:size()

In [ ]:
70/156